In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import requests
#from selenium import webdriver
import time

In [2]:
import tqdm
import time
import json
import pandas as pd

In [3]:
#r = requests.get('https://aibee.com.tw/address/')
#r.encoding = "UTF-8"
#soup = BeautifulSoup(r.text, 'html.parser')
basic = 'https://aibee.com.tw'
basic_register = 'https://aibee.com.tw/br/'
basic_address = 'https://aibee.com.tw/address/'

In [4]:
def get_link_register(basic) :
    
    ## grasp page
    r = requests.get(basic+'/br/')
    r.encoding = "UTF-8"
    soup = BeautifulSoup(r.text, 'html.parser')
    
    ## get all url (company and commercial)
    
    
    ## get page list
    page_list = list(map(lambda x:x.get('href'),soup.find(class_ ='NavPg p0').find_all('a')))[:-2]  ## <-- cut first and last
    
    title_register_company = []
    href_register_company = []
    title_register_commercial = []
    href_register_commercial = []
    for page in page_list :
    
        r = requests.get(basic+page)
        r.encoding = "UTF-8"
        soup = BeautifulSoup(r.text, 'html.parser')     
    
        link_company = soup.find_all(class_ = 'p0')[0].find_all('a')
        title_register_company.extend(list(map(lambda x :x.get('title'),link_company)))
        href_register_company.extend(list(map(lambda x :basic + x.get('href'),link_company)))
        
        link_commercial = soup.find_all(class_ = 'p0')[1].find_all('a')
        title_register_commercial.extend(list(map(lambda x :x.get('title'),link_commercial)))
        href_register_commercial.extend(list(map(lambda x :basic + x.get('href'),link_commercial)))    
        
        
    register_company = dict(zip(title_register_company,href_register_company))
    register_commercial = dict(zip(title_register_commercial,href_register_commercial))
    return register_company , register_commercial

def get_link_address (basic) :
    
    r = requests.get(basic + '/address/')
    r.encoding = "UTF-8"
    soup = BeautifulSoup(r.text, 'html.parser')
    
    title_address_city = []
    href_address_city = []
    for city in range(len(soup.find(class_ = 'content').find_all('li'))) :
        
        link_address = soup.find(class_ = 'content').find_all('li')[city].find_all('a')

        title_address_city.extend(list(map(lambda x:x.get('title'),link_address)))
        href_address_city.extend(list(map(lambda x:basic + x.get('href'),link_address)))
        
    address_city_link = dict(zip(title_address_city,href_address_city))
    
    address_city_link = pd.DataFrame.from_dict(address_city_link,orient='index').reset_index()
    address_city_link.columns = ['address_city','address_url']
    
    return address_city_link

def get_link_address_sub (basic,address_city_link_title,address_city_link_url) :

    address_city_name = address_city_link_title
    r = requests.get(address_city_link_url)
    r.encoding = "UTF-8"
    soup = BeautifulSoup(r.text, 'html.parser')

    total_company = soup.find(class_ = 'content').find('h2').text
    
    basic_page = soup.find(class_ ='NavPg p0').find_all('a')[0].get('href')
    
    try :
        page = (int(soup.find(class_ ='NavPg p0').find_all('a')[-1].get('href')[-4:-1]) -1)  /20
    except TypeError:
        page = (int(soup.find(class_ ='NavPg p0').find_all('a')[-1].get('href')[-3:-1]) -1)  /20

    title_address_city_town = []
    href_address_city_town = []
    for i in range(0,int(page+1)) :
        if i == 0 :
            r = requests.get(basic + basic_page)
            r.encoding = "UTF-8"
            soup = BeautifulSoup(r.text, 'html.parser')
            
            title_address_city_town.extend(list(map(lambda x:x.find('a').get('title'),soup.find_all('li',class_='p0 BbHr'))))
            href_address_city_town.extend(list(map(lambda x:basic + x.find('a').get('href'),soup.find_all('li',class_='p0 BbHr'))))
            
        else :
            r = requests.get(basic + basic_page  +str(i*20 + 1)+'/')
            r.encoding = "UTF-8"
            soup = BeautifulSoup(r.text, 'html.parser')
            title_address_city_town.extend(list(map(lambda x:x.find('a').get('title'),soup.find_all('li',class_='p0 BbHr'))))
            href_address_city_town.extend(list(map(lambda x:basic + x.find('a').get('href'),soup.find_all('li',class_='p0 BbHr'))))
            
    
    city_company = dict(zip(title_address_city_town,href_address_city_town))
    
    city_company = pd.DataFrame.from_dict(city_company,orient='index').reset_index()
    city_company.columns = ['city_company','city_company_url']
    city_company['total_company'] = total_company
    city_company['address_city'] = address_city_name
    
    return city_company

def get_detail_address_sub (basic,address_city_name,city_company_title,city_company_url) :
    
    ### get 資本額等等 
    
    address_city = address_city_name
    city_company = city_company_title
    
    r = requests.get(city_company_url)  ### address_city_town is the url of the company !! 
                                         ### example : 'https://aibee.com.tw/v/26747432/'
    r.encoding = "UTF-8"
    soup = BeautifulSoup(r.text, 'html.parser')    
    
    key = []
    val = []
    count = 1
    for content in soup.find('div',class_='content').find_all('span') :

        if (count % 2 != 0 and count <=15):
            key.append(content.text)

        else :
            val.append(content.text)

        count += 1
    
    count_category = 0
    count_industry = 0
    for content in soup.find('div',class_='content').find_all('a') :

        if ('category' in content.get('href') and count_category <=10)  :
            key.append(f'category{count_category}')
            val.append(content.text)
            count_category +=1
        elif ('industry' in content.get('href') and count_industry <=10) :
            key.append(f'industry{count_industry}')
            val.append(content.text)
            count_industry +=1
        else :
            pass
        
    company_detail = dict(zip(key,val))    
    
    company_detail = pd.DataFrame.from_dict(company_detail,orient='index').T 
    company_detail['address_city'] = address_city
    company_detail['city_company'] = city_company
    
    return company_detail

### 轉成 List 即可直接轉為dataframe

In [5]:
def crawler_all_city_company(basic,city) :  ### city rules is  '台中市|彰化縣|....|'
    
    df_city_link = get_link_address(basic)
    df_city_link_filter = df_city_link[df_city_link['address_city'].str.contains(city,na=False)]
    
    
    df_city_company_all = pd.DataFrame()
    df_company_detail_all = pd.DataFrame()
    
    print('Start crawler {}'.format(city))
    pbar1 = tqdm.tqdm(total=len(df_city_link_filter))
    temp = []
    for city_name , city_url in zip(df_city_link_filter['address_city'],df_city_link_filter['address_url']) :
        
        df_city_company = get_link_address_sub(basic,city_name,city_url)
        df_city_company_all = pd.concat([df_city_company_all,df_city_company])
        pbar1.update(1)
        time.sleep(0.01)
        print('finish {}'.format(city_name))
        temp.append(city_name)
    pbar1.close()
    print('Done')
    
    print('Strat crawler {}'.format(temp))
    pbar2 = tqdm.tqdm(total=len(df_city_company_all))
    for address_city,city_company,company_url in zip(df_city_company_all['address_city'],df_city_company_all['city_company']
                           ,df_city_company_all['city_company_url']) :
        
        df_company_detail = get_detail_address_sub(basic,address_city,city_company,company_url)
        df_company_detail_all = pd.concat([df_company_detail_all,df_company_detail])
        pbar2.update(1)
        time.sleep(0.01)

    pbar2.close()
    
    print('Done')
    
    return df_city_link_filter , df_city_company_all , df_company_detail_all

In [7]:
temp1,temp2,temp3 = crawler_all_city_company(basic,'基隆市仁愛區|基隆市信義區|基隆市中正區')


  0%|                                                                                            | 0/3 [00:00<?, ?it/s]

Start crawler 基隆市仁愛區|基隆市信義區|基隆市中正區



 33%|████████████████████████████                                                        | 1/3 [00:09<00:18,  9.15s/it]

finish 基隆市仁愛區公司行號



 67%|████████████████████████████████████████████████████████                            | 2/3 [00:21<00:10, 10.22s/it]

...

 88%|████████████████████████████████████████████████████████████████████▋         | 2640/3000 [12:04<01:40,  3.59it/s]


 92%|████████████████████████████████████████████████████████████████████████      | 2772/3000 [12:47<01:03,  3.62it/s]


 97%|███████████████████████████████████████████████████████████████████████████▌  | 2904/3000 [13:27<00:31,  3.08it/s]


100%|██████████████████████████████████████████████████████████████████████████████| 3000/3000 [13:51<00:00,  3.61it/s]

Done


In [9]:
temp1.to_csv('../data/temp1.csv')
temp2.to_csv('../data/temp2.csv')
temp3.to_csv('../data/temp3.csv')